In [4]:
import pandas as pd

In [5]:
train = pd.read_csv('./Dacon/funda_train.csv')

In [6]:
train.head(6)

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000
5,0,5,c,2016-06-02,13:09,0,NaN,기타 미용업,2000.000000


## 처음 추정치

In [47]:
set_amount = train[(train['transacted_date']>='2018-03-01') & (train['transacted_date'] <= '2018-05-31')]

In [85]:
main_expectations = set_amount.groupby('store_id')['amount'].sum()

In [86]:
main_expectations

store_id
0       2.171000e+06
1       2.977143e+05
2       9.461429e+05
4       2.475643e+06
5       1.111857e+06
            ...     
2132    2.518143e+06
2133    1.418629e+06
2134    5.340714e+05
2135    1.491000e+06
2136    6.743929e+06
Name: amount, Length: 1955, dtype: float64

## 새로운 방법
### 1. (2019년 1월~2월까지의 매출의 합/ 59(일))*92 
* 최근 두달동안의 일 평균 매출을 구해서 19.03.01 ~ 19.05.31 사이의 날짜만큼 매출로 추정해본다.

### 2. 92일 동안의 요일별 갯수를 구해서 각 스토어의 요일별 매출을 날짜 수로 나눠서 구해본다.






















































































































































































































### + 모두 다 한 다음에 전에 구했던 추정치 결과값에 붙여준다.(union 사용)

# 잃어버린 데이터를 찾아서

In [8]:
set_amount = train[(train['transacted_date']>='2018-03-01') & (train['transacted_date'] <= '2018-05-31')]

In [9]:
missing_datas = train[~(train['store_id'].isin(set_amount['store_id']))]

In [62]:
missing_datas

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
779947,271,522127,a,2016-06-01,13:05,0,인천 남구,NaN,2285.714286
779948,271,522128,f,2016-06-01,13:06,0,인천 남구,NaN,8414.285714
779949,271,522129,g,2016-06-01,13:08,0,인천 남구,NaN,1350.000000
779950,271,522130,d,2016-06-01,13:15,0,인천 남구,NaN,1428.571429
779951,271,522131,b,2016-06-01,14:38,0,인천 남구,NaN,6285.714286
779952,271,522132,c,2016-06-01,15:17,0,인천 남구,NaN,214.285714
779953,271,522133,b,2016-06-01,15:56,0,인천 남구,NaN,7014.285714
779954,271,522134,b,2016-06-01,16:34,0,인천 남구,NaN,10000.000000
779955,271,522135,a,2016-06-01,16:36,0,인천 남구,NaN,5714.285714
779956,271,522136,c,2016-06-01,16:46,0,인천 남구,NaN,4285.714286


In [68]:
missing_id = missing_datas['store_id'].unique()

## 1. (2019년 1월~2월까지의 매출의 합/ 59(일))*92

In [15]:
train

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000
5,0,5,c,2016-06-02,13:09,0,NaN,기타 미용업,2000.000000
6,0,6,f,2016-06-02,15:33,0,NaN,기타 미용업,2000.000000
7,0,7,a,2016-06-02,17:18,0,NaN,기타 미용업,7857.142857
8,0,8,c,2016-06-02,18:30,0,NaN,기타 미용업,2000.000000
9,0,9,a,2016-06-02,19:56,0,NaN,기타 미용업,1857.142857


In [35]:
missing_stores = missing_datas[(missing_datas['transacted_date'] >= '2019-01-01')&(missing_datas['transacted_date'] <= '2019-02-29')]

In [36]:
missing_stores

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
785990,271,524967,b,2019-01-03,12:52,0,인천 남구,NaN,8.142857e+03
785991,271,524968,f,2019-01-04,21:35,5,인천 남구,NaN,7.142857e+05
785992,271,524969,b,2019-01-04,21:36,5,인천 남구,NaN,5.714286e+05
785993,271,524970,a,2019-01-04,21:37,5,인천 남구,NaN,6.428571e+05
785994,271,524971,a,2019-01-04,21:39,5,인천 남구,NaN,1.071429e+06
785995,271,524972,g,2019-01-04,21:41,5,인천 남구,NaN,4.714286e+05
785996,271,524973,h,2019-01-04,21:43,5,인천 남구,NaN,2.700000e+05
785997,271,524935,c,2019-01-04,21:44,5,인천 남구,NaN,6.471429e+05
785998,271,524969,b,2019-01-04,21:47,0,인천 남구,NaN,1.056571e+03
785999,271,524974,e,2019-01-05,16:08,10,인천 남구,NaN,3.392446e+05


In [110]:
missing_stores.groupby('store_id')['amount'].sum()

store_id
271     1.024661e+07
279     2.295000e+05
515     6.595714e+04
588     3.897143e+04
706     4.069143e+05
795     0.000000e+00
1379    1.016071e+06
1435    7.361429e+05
2081    8.412086e+05
2094    2.816429e+05
Name: amount, dtype: float64

In [94]:
missing_expectation = missing_stores.groupby('store_id')['amount'].sum()/59*92

In [95]:
missing_expectation

store_id
271     1.597776e+07
279     3.578644e+05
515     1.028484e+05
588     6.076901e+04
706     6.345104e+05
795     0.000000e+00
1379    1.584383e+06
1435    1.147884e+06
2081    1.311715e+06
2094    4.391719e+05
Name: amount, dtype: float64

## 합쳐보자

In [96]:
main_expectations

store_id
0       2.171000e+06
1       2.977143e+05
2       9.461429e+05
4       2.475643e+06
5       1.111857e+06
            ...     
2132    2.518143e+06
2133    1.418629e+06
2134    5.340714e+05
2135    1.491000e+06
2136    6.743929e+06
Name: amount, Length: 1955, dtype: float64

In [97]:
missing_expectation

store_id
271     1.597776e+07
279     3.578644e+05
515     1.028484e+05
588     6.076901e+04
706     6.345104e+05
795     0.000000e+00
1379    1.584383e+06
1435    1.147884e+06
2081    1.311715e+06
2094    4.391719e+05
Name: amount, dtype: float64

In [98]:
result = pd.concat([main_expectastions, missing_expectation])

In [99]:
result.to_csv('./Dacon/second_test.csv')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


### missing 된 것들이 있는지 알아보는 법

In [108]:
len(missing_id)

12

In [80]:
missing_stores['store_id'].isin(list(missing_id))

785990     True
785991     True
785992     True
785993     True
785994     True
           ... 
6398249    True
6398250    True
6398251    True
6398252    True
6398253    True
Name: store_id, Length: 1371, dtype: bool

##  92일 동안의 요일별 갯수를 구해서 각 스토어의 요일별 매출을 날짜 수로 나눠서 구해본다.

AttributeError: module 'pandas' has no attribute 'DataFram'